In [88]:
# Test the model
# import the necessary libraries

import numpy as np
import pandas as pd
import joblib

In [89]:
# Load the model
model = joblib.load('../models/genre_model_based_number_of_rating.pkl')

# Load the encoder
age_encoder = joblib.load('../models/age_ohe.pkl')
occupation_encoder = joblib.load('../models/occupation_ohe.pkl')

# Load the genre columns
genre_columns = joblib.load('../models/genre_columns.pkl')

# Load the age map
age_map = joblib.load('../models/age_map.pkl')

In [90]:
# Define the function for age range mapping
def age_map_convertor(age):
    if age <= 18:
        return 1
    elif age <= 24:
        return 18
    elif age <= 34:
        return 25
    elif age <= 44:
        return 35
    elif age <= 49:
        return 45
    elif age <= 55:
        return 50
    else:
        return 56

# Define the function to prepare the data
def prepare_data(age, occupation, gender):
    # Map the age to the corresponding range key
    age_str = age_map[age_map_convertor(age)]

    # Ensure that age and occupation are in 2D format
    age_input = np.array(age_str).reshape(1, -1)  # Convert age to a 2D array
    occupation_input = np.array(occupation).reshape(1, -1)  # Convert occupation to a 2D array

    # Transform the data using the encoder
    age_encoded = age_encoder.transform(age_input).toarray()  # Convert sparse matrix to dense array
    occupation_encoded = occupation_encoder.transform(occupation_input).toarray()  # Convert sparse matrix to dense array

    # Combine all features into a single array
    input_features = np.hstack([gender, age_encoded.flatten(), occupation_encoded.flatten()])

    # Reshape input to 2D array as expected by the model
    input_features = input_features.reshape(1, -1)

    feature_names = ['gender'] + list(age_encoder.get_feature_names_out(['AgeRange'])) + list(occupation_encoder.get_feature_names_out(['Occupation']))
    input_features_df = pd.DataFrame(input_features, columns=feature_names)
    
    return input_features_df

In [95]:
# Prepare a sample input
gender = 1  # 0 for Male, 1 for Female
age = 45  # Age 
occupation = "lawyer"  # Occupation code for "college/grad student"

In [96]:
# Prepare the input data
input_features = prepare_data(age, occupation, gender)

In [97]:
# Make predictions
predictions = model.predict(input_features)

In [98]:
# Convert binary predictions to genre labels
predicted_genre_labels = [genre for genre, flag in zip(genre_columns, predictions[0]) if flag == 1]

# Display the predicted genres
print("Predicted Genres:", predicted_genre_labels)

Predicted Genres: ['Comedy', 'Drama', 'Thriller']
